Ucitavamo dataset

In [2]:
import torch
import numpy as np 
from torch.utils.data import TensorDataset



ulaz= np.loadtxt('TrainDataset.txt')
podaci=ulaz[:,1:]
obelezja=ulaz[:,:1]
obelezja=torch.Tensor(obelezja)
obelezja=obelezja.flatten()

ulaz= np.loadtxt('TestDataset.txt')
tpodaci=ulaz[:,1:]
tmp = np.concatenate((podaci,tpodaci))
b=np.max(tmp,axis=0)
podaci=podaci/b
tpodaci=tpodaci/b
tobelezja=ulaz[:,:1]
tpodaci=torch.Tensor(tpodaci)
tobelezja=torch.Tensor(tobelezja)
tobelezja=tobelezja.flatten()


podaci=torch.Tensor(podaci)

test_set=TensorDataset(tpodaci,tobelezja)
dataset=TensorDataset(podaci,obelezja)
train_set,val_set = torch.utils.data.random_split(dataset,[0.9,0.1])

In [3]:
print(len(train_set))
print(len(val_set))
print(len(test_set))
print(len(b))

112500
12500
24998
1564


In [4]:
print(test_set[24100])

(tensor([0.5020, 0.4962, 0.4383,  ..., 0.0000, 0.0536, 0.5730]), tensor(48.))


In [5]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    print("NE RADI CUDA!")

NE RADI CUDA!


Inicijalizacija mreze

In [6]:
import torch.nn as nn

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.act=nn.Sequential(
            nn.Linear(1564,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(1000,300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300,120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120,50),
            nn.BatchNorm1d(50),
            nn.ReLU()
        )
        self.act=self.act.to(device)
    def forward (self, x):
        return self.act(x)
model = Mreza()
model = model.to(device)

Testiranje

In [7]:
def totensor(y):
    ret=np.zeros((len(y),50))
   # print(y[0])
   # print(y[0][0])
   # print(y[0][0].item())
    for i in range(0,len(y)):
        ret[i][round(y[i].item())]=1
    return torch.Tensor(ret).to(device)

def test(model,sample):
    #print(sample)
    sample=sample.to(device)
    rez=model(sample.reshape(1,1564))
    _,b=torch.max(rez,dim=1)
    return b


Inicijalizacija parametara

In [8]:
broj_epoha = 15
batch_velicina=500
loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=val_set,batch_size=batch_velicina,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)


In [9]:
def get_accuracy(y_hat,y):
   # print(y)
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    y_hat=np.argmax(y_hat,axis=1)
   # print(y_hat,y)
    corrects=(y_hat==y).sum()
   # print(corrects)
    return corrects.item()

def validation(model):
    with torch.no_grad():
        model.eval()
        kol=0
        total_loss=0
        for batch_num, (x, y) in enumerate(val_loader):
                x = x.to(device)
                y = y.to(device)
                #print(y)
                y_hat = model(x)
                loss=loss_funk(y_hat,totensor(y))
                total_loss+=loss.detach().item()
                kol+=get_accuracy(y_hat=y_hat, y=y)/len(y)
                del y_hat
        kol=kol*100/batch_num
        total_loss/=batch_num
    return total_loss,kol

Treniranje

In [11]:
# MAKE SURE TO INITIALIZE PARAMETERS BEFORE STARTING TRAINING
best_model=model
best_acc=0

import copy
import time
import datetime
ime="TrainingLogs\\FC_TRAINING_"
ime+=datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
ime+=".txt"
#print(ime)
ouf = open(ime,"w")  
for epoha in range(1,broj_epoha+1):
      train_running_loss =0.0
      train_acc=0.0
      model = model.train() 
      start=time.time()
      for batch_num, (x, y) in enumerate(loader):
            x = x.to(device)
            y = y.to(device)
           # print(y)
            #print(y)
            y_hat = model(x)
            loss = loss_funk(y_hat, totensor(y))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)/len(y)
            del y_hat

            print(f'Epoch: {epoha}\t{100 * (batch_num + 1) / len(loader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch. Time until completion {(time.time()-start)/((batch_num + 1) / len(loader)) - time.time()+start:.2f} seconds.',
                end='\r')

      print(f'',end='\n')    
      epoch_loss = train_running_loss / batch_num
      epoch_acc = 100*train_acc / batch_num
      val_loss,val_acc = validation(model)
      if val_acc>best_acc:
            best_acc=val_acc
            best_model=copy.deepcopy(model)
            torch.save(best_model,'FCBestModel.pt')
            ouf.write("NEW BEST MODEL: ")
            print("NEW BEST MODEL: ",end='')
      print('Epoch: %d | Train Loss: %.4f | Train Accuracy: %.2f | Validation Loss: %.4f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc,val_loss,val_acc))

Epoch: 1	100.00% complete. 46.01 seconds elapsed in epoch. Time until completion 0.00 seconds.
NEW BEST MODEL: Epoch: 1 | Train Loss: 3.4365 | Train Accuracy: 13.98 | Validation Loss: 3.4164 | Validation Accuracy: 16.59
Epoch: 2	100.00% complete. 43.84 seconds elapsed in epoch. Time until completion 0.00 seconds.
NEW BEST MODEL: Epoch: 2 | Train Loss: 3.0544 | Train Accuracy: 21.66 | Validation Loss: 3.2064 | Validation Accuracy: 20.99
Epoch: 3	100.00% complete. 42.68 seconds elapsed in epoch. Time until completion 0.00 seconds.
NEW BEST MODEL: Epoch: 3 | Train Loss: 2.8391 | Train Accuracy: 26.32 | Validation Loss: 3.1240 | Validation Accuracy: 22.98
Epoch: 4	100.00% complete. 42.63 seconds elapsed in epoch. Time until completion 0.00 seconds.
NEW BEST MODEL: Epoch: 4 | Train Loss: 2.6582 | Train Accuracy: 30.55 | Validation Loss: 3.0840 | Validation Accuracy: 24.00
Epoch: 5	100.00% complete. 42.79 seconds elapsed in epoch. Time until completion 0.00 seconds.
NEW BEST MODEL: Epoch: 5 

In [ ]:
def testt(model):
        kol=0
        for i in range(0,len(val_set)):
            out=test(model,val_set[i][0].reshape(1,1564))
            ans=val_set[i][1].item()
            if round(ans)==out:
                kol+=1
        return kol
print(testt(model))
print(testt(model)/len(val_set)*100)

In [12]:
print(best_acc)
print(validation(best_model))

26.808333333333334
(3.0317836304505668, 26.808333333333334)


### OBRADA REZULTATA

In [35]:
gmatrix = []
pokusaj = []
for i in range(0,len(test_set)):
    a=test_set[i][0].to(device)
    rez=model(a.reshape(1,1564))
    _,b=torch.max(rez,dim=1)
    gmatrix.append(b.item())
    npa=rez.cpu().reshape(50).detach().numpy()
    pok=np.sum(npa>npa[round(test_set[i][1].item())])+1
    pokusaj.append(pok)
print(pokusaj)
gmatrix=np.array(gmatrix)
labele=[]
for i in range(0,len(test_set)):
    labele.append(test_set[i][1].item())
labele=np.array(labele)
print(gmatrix,labele)

[3, 12, 2, 1, 3, 8, 1, 10, 5, 1, 2, 1, 30, 31, 7, 2, 27, 6, 2, 6, 2, 11, 17, 29, 3, 6, 3, 14, 1, 3, 9, 6, 30, 12, 33, 7, 1, 2, 31, 33, 3, 3, 9, 3, 3, 4, 3, 1, 6, 1, 8, 4, 1, 21, 3, 2, 9, 4, 25, 5, 12, 14, 1, 6, 3, 31, 4, 14, 2, 3, 1, 3, 2, 1, 35, 2, 13, 11, 2, 7, 1, 1, 2, 1, 6, 5, 9, 16, 7, 3, 6, 1, 2, 8, 2, 6, 11, 3, 6, 8, 30, 2, 6, 2, 9, 6, 2, 3, 8, 28, 8, 8, 6, 3, 3, 5, 22, 2, 7, 2, 6, 2, 6, 1, 4, 26, 5, 2, 3, 5, 3, 1, 3, 5, 30, 2, 2, 6, 11, 9, 19, 2, 9, 15, 2, 1, 1, 1, 1, 3, 2, 6, 1, 2, 4, 3, 4, 3, 14, 1, 1, 5, 10, 2, 3, 1, 5, 3, 3, 29, 1, 22, 9, 1, 2, 5, 3, 34, 2, 11, 8, 2, 2, 4, 6, 12, 2, 1, 6, 1, 4, 4, 25, 8, 12, 5, 1, 16, 3, 1, 5, 1, 3, 2, 30, 1, 4, 10, 6, 2, 3, 21, 8, 20, 4, 4, 6, 3, 13, 10, 2, 9, 2, 3, 7, 3, 3, 4, 1, 4, 8, 9, 3, 7, 12, 3, 1, 2, 3, 14, 5, 1, 2, 7, 26, 13, 1, 8, 1, 5, 1, 3, 2, 5, 4, 10, 11, 1, 2, 34, 3, 10, 1, 4, 11, 1, 1, 14, 6, 4, 1, 1, 1, 11, 1, 9, 4, 1, 5, 5, 3, 1, 3, 2, 9, 7, 28, 19, 12, 2, 1, 17, 19, 17, 2, 9, 4, 8, 12, 5, 2, 1, 2, 1, 7, 5, 24, 2, 1, 6, 3

In [19]:
states=[
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
    'Wisconsin', 'Wyoming'
]

In [14]:


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(labele, gmatrix)

for i in range(0, 50):
    p = np.sum(cf_matrix[i])
    cf_matrix[i] = cf_matrix[i] * 100 / p

plt.figure(figsize=(160, 90))
sns.heatmap(cf_matrix/100, annot=True, fmt='.2%', cmap='Blues')
plt.show()

In [24]:
state_tacnost=np.zeros((50,2))
for i in range(0,50):
    state_tacnost[i][0]=i
    state_tacnost[i][1]=cf_matrix[i][i]
state_tacnost=state_tacnost[state_tacnost[:,1].argsort()]
#print(state_tacnost)
for i in range(0,50):
    print(states[round(state_tacnost[i][0])],state_tacnost[i][1],(100-state_tacnost[i][1])/100*255,state_tacnost[i][1]/100*255,0)

Michigan 6.0 239.7 15.299999999999999 0
Virginia 7.0 237.15 17.85 0
Pennsylvania 8.0 234.60000000000002 20.400000000000002 0
Illinois 8.0 234.60000000000002 20.400000000000002 0
Ohio 8.0 234.60000000000002 20.400000000000002 0
Connecticut 9.0 232.05 22.95 0
Missouri 10.0 229.5 25.5 0
Wisconsin 11.0 226.95000000000002 28.05 0
Louisiana 12.0 224.4 30.599999999999998 0
South Carolina 13.0 221.85 33.15 0
Arkansas 13.0 221.85 33.15 0
Kentucky 14.0 219.29999999999998 35.7 0
Texas 14.0 219.29999999999998 35.7 0
Massachusetts 14.0 219.29999999999998 35.7 0
Kansas 14.0 219.29999999999998 35.7 0
Colorado 14.0 219.29999999999998 35.7 0
Tennessee 15.0 216.75 38.25 0
North Carolina 15.0 216.75 38.25 0
Alabama 15.0 216.75 38.25 0
Georgia 15.0 216.75 38.25 0
Indiana 15.0 216.75 38.25 0
New York 16.0 214.2 40.800000000000004 0
New Jersey 17.0 211.64999999999998 43.35 0
Iowa 18.0 209.1 45.9 0
Oregon 18.0 209.1 45.9 0
Washington 19.0 206.55 48.45 0
Rhode Island 20.0 204.0 51.0 0
Florida 20.0 204.0 51.0 

In [25]:
import matplotlib.pyplot as plt
from geopandas

# Sample dictionary with state colors
state_colors = {
    'Alabama': 'red',
    'Alaska': 'blue',
    'Arizona': 'green',
    'Arkansas': 'orange',
    # Add more states and colors here...
}

def plot_us_states(state_colors):
    # Create a Basemap of the US
    plt.figure(figsize=(12, 9))
    m = Basemap(projection='merc', llcrnrlat=25, urcrnrlat=49,
                llcrnrlon=-125, urcrnrlon=-66, resolution='l')

    # Draw state boundaries
    m.drawcoastlines()
    m.drawcountries(linewidth=2)
    m.drawstates()

    # Get state boundaries and color them
    for state, color in state_colors.items():
        try:
            state_info = m.states[state]
            state_shape = state_info[0]
            segs = state_shape[0]
            poly = plt.Polygon(segs, edgecolor='black', facecolor=color)
            plt.gca().add_patch(poly)
        except KeyError:
            print(f"Warning: {state} is not recognized as a valid state name.")

    # Show the map
    plt.title("Colored US States")
    plt.show()
plot_us_states(state_colors)

ImportError: cannot import name 'Basemap' from 'geopandas' (c:\Users\VladaM\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\__init__.py)